In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np 
import pandas as pd 
import json
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm

In [0]:
%time
from zipfile import ZipFile

downloaded_train = drive.CreateFile({'id':'10qtOKBsGFQlZ6YzQxV30uHT7truULE1E'}) 
downloaded_train.GetContentFile('train.csv.zip')

with ZipFile('train.csv.zip', 'r') as zipObj:
   zipObj.extractall()

down_main_cat_map = drive.CreateFile({'id': '1l70MFuRBb7IgaVn6hrx0_X3lU0_q0B7C'}) 
down_main_cat_map.GetContentFile('main_category_mapper.json')

downloaded_unsupervised = drive.CreateFile({'id':'1vTeZFDl4ugkRv_6mUCl7gJk8DMnE8ECB'}) 
downloaded_unsupervised.GetContentFile('unsupervised.csv.zip')

with ZipFile('unsupervised.csv.zip', 'r') as zipObj:
   zipObj.extractall()

down_sub_cat_map = drive.CreateFile({'id': '1SJhd1yLU2V6eKGKI_R7Gw0ybGC9RaxJu'}) 
down_sub_cat_map.GetContentFile('sub_category_mapper.json')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [0]:
train = pd.read_csv('train.csv')

In [0]:
downloaded_sub = drive.CreateFile({'id':'1QqoCIhAJlMxW5_TSaNnOUU08civyHWOP'}) 
downloaded_sub.GetContentFile('test.csv')

In [0]:
test = pd.read_csv('test.csv')
test.head()

,index,question
0,0,"Подскажите сайт российский, вещи заказывать: )..."
1,1,Как заказать Рикардо Милоса в Москве?
2,2,"Пустят ли меня в Израиль, если я работаю в ОАЭ?"
3,3,почему о наличии пальмового масла и консервато...
4,4,Вот во всех бедах России винят правительство.....


In [0]:
import re
def filter_ru_en_words(x):
  x = ' '.join(re.findall('[a-zа-яё0-9]+', x.lower()))
  x = re.sub('[0-9]+', ' число ', x)
  x = re.sub('ё', 'е', x)
  x = re.sub('\s+', ' ', x)
  return x
train['w_cl'] = train.question.apply(filter_ru_en_words)

In [0]:
!pip install pymorphy2

In [0]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
words = [j for i in train.w_cl.tolist() for j in i.split(' ')]
to_norm_form = { i: morph.parse(i)[0].normal_form for i in list(set(words))}

print('Vocab_size :', len(list(set(to_norm_form.values()))))

Vocab_size : 157775


In [0]:
def normalize(x):
  return ' '.join([to_norm_form[i] if i in to_norm_form else i for i in x.split(' ')])

train['norm_question'] = train.w_cl.apply(normalize)

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [0]:
!pip install transformers

     |████████████████████████████████| 389kB 6.9MB/s 
     |████████████████████████████████| 1.0MB 45.2MB/s 
     |████████████████████████████████| 860kB 40.4MB/s 
     |████████████████████████████████| 675kB 41.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=66033d58855fd1511909a71421b7b19ac9ad1a23534e25324a910ab00cb0b7ad
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609171 sha256=82d62c11abc6c96db68c7a21956ba0b5d592b05b1af41f97584cb6043c34de03
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built sacremoses regex


In [0]:
import torch
from transformers import *

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [0]:
train.head()

,main_category,sub_category,question
0,4,32,Законно ли изменение количества дней листа нет...
1,1,8,"Погадайте что будут, у меня предчувствия?) К ч..."
2,10,66,Какой дровокол надёжней?
3,6,45,Зачем в старой Японии женщины бинтовали грудь ...
4,6,47,"Положив руку.. на ...ну что там у вас, мужчин"


In [0]:
def process_text(text):

    words = wordpunct_tokenize(text)

    return words

In [0]:
import random

In [0]:
from torch.utils.data import Dataset
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, sequence_length=64, verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.sequence_length = sequence_length
        
        self.pad_index = 100
        self.m = 0 
        
        self.load(x_data, verbose=verbose)
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            text = text.lower()
            words = process_text(text)
            index = random.choice([i for i in range(len(words))])
            words[index] = '[MASK]'
            text = ' '.join(words)
            text = "[CLS] " +  text + " [SEP]"
            words = tokenizer.tokenize(text)
            indexed_words = tokenizer.convert_tokens_to_ids(words)
            if len(indexed_words)>self.m:
              self.m = len(indexed_words)
            self.x_data.append(indexed_words)
        

    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        
        ### CODE ###

        if len(sequence) > self.sequence_length:
            sequence = sequence[:self.sequence_length]
        elif len(sequence) < self.sequence_length:
            sequence = sequence + [100] * (self.sequence_length - len(sequence))

        return sequence
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]

        if len(x)<=self.sequence_length and len(x)>0:
          len_x = len(x)
        elif len(x)<=0:
          len_x=1
        else:
          len_x = self.sequence_length
        
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y, len_x, torch.tensor([1]*self.sequence_length)

In [0]:
train = train[:5000]
test = test[:5000]

In [0]:
#tokens_tensor = torch.tensor([indexed_tokens])
#segments_tensors = torch.tensor([segments_ids])


model = BertModel.from_pretrained('bert-base-multilingual-uncased')


In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [0]:
x_train, x_validation, y_train, y_validation = train_test_split(train.question, train.main_category, test_size=0.1)

train_dataset = WordData(list(x_train), list(y_train))
train_loader = DataLoader(train_dataset, batch_size=512)

validation_dataset = WordData(list(x_validation), list(y_validation))
validation_loader = DataLoader(validation_dataset, batch_size=512)

test_dataset = WordData(list(test.question), np.zeros((test.shape[0])))
test_loader = DataLoader(test_dataset, batch_size=512 )

Loading data: 100%|██████████| 200000/200000 [01:20<00:00, 2491.97it/s]


In [0]:
for x, y, len_x, ones in train_loader:
  print(ones.shape)
  break

torch.Size([512, 64])


In [0]:
y

tensor([11,  8, 22, 16, 16, 27, 20,  3,  8, 12, 25, 12,  3, 16,  1,  2, 27, 18,
        11, 19, 13,  1, 13, 18, 12, 13, 26,  2, 13,  2,  2,  2, 24, 14,  2,  3,
         6, 13, 20, 11,  4, 19, 20, 27, 27, 27,  8, 16, 20,  0, 20,  0, 21,  7,
         7, 24, 11, 16,  3, 20,  8, 15, 21,  2, 22, 20, 12, 27, 10, 26,  5, 25,
        15, 10, 16, 20, 14, 21,  5, 26, 26, 26, 22, 20, 13, 17,  2,  7, 26, 13,
         1, 14, 27, 22, 13, 19, 19, 19, 13, 13,  5,  5,  8,  8,  5, 16, 26, 25,
        15, 20, 25, 14,  5, 10,  6, 12,  0, 27,  9, 25, 10, 24, 17, 15,  6, 20,
        18, 10, 20, 19, 26,  8,  0, 16,  0, 24,  5,  5, 25, 24, 21, 22,  5, 24,
        19, 10, 13, 19, 18, 20,  9,  1, 16,  5,  0, 20, 18, 16,  8, 14,  1, 19,
        19,  9, 11, 24,  4, 22, 17,  4, 14, 19,  4, 15, 12,  0, 13,  5,  2, 19,
         0,  2, 22, 19,  0,  7,  6, 14,  8,  6,  7, 16,  9, 18,  9,  2,  5,  7,
        10, 15,  1, 19, 20,  4, 24,  0,  0, 20,  9, 16,  9,  8,  0, 26, 13, 27,
         8, 11, 27, 17,  3, 19, 16, 27, 

In [0]:
n_classes = train.main_category.unique().shape[0]

In [0]:
from math import sqrt
class MLP(torch.nn.Module):
    
    def __init__(self, in_features, inner_features, inner_features2, out_features):
        
        # Главная задача этого метода - возможность использования в классе потомке, методов класса-родителя
        # Вызываем __init__ родителя - torch.nn.Module
        super().__init__()
        
        # определяем слои и все что захотим сохранять/использовать
        self.linear_1 = torch.nn.Linear(in_features=in_features, out_features=inner_features)
        gl = sqrt(6./(in_features+inner_features))
        self.linear_1.weight.data.uniform_(-gl, gl)
        self.non_linear_function = torch.nn.ReLU()
        self.drop_out = torch.nn.Dropout(0.18)
        self.linear_2 = torch.nn.Linear(in_features=inner_features, out_features=inner_features2)
        gl1 = sqrt(6./(inner_features+inner_features2))
        self.linear_2.weight.data.uniform_(-gl1, gl1)
        self.non_linear_function2 = torch.nn.ReLU()
        self.linear_3 = torch.nn.Linear(in_features=inner_features2, out_features=out_features)
        gl2 = sqrt(6./(inner_features2+out_features))
        self.linear_3.weight.data.uniform_(-gl2, gl2)
  
    def forward(self, x):
        
        # наша привычная схема
        x = self.linear_1(x)
        x = self.non_linear_function(x)
        x = self.drop_out(x)
        x = self.linear_2(x)
        x = self.non_linear_function2(x)
        x = self.linear_3(x)
        return x

In [0]:
mlp = MLP(in_features=768, inner_features=(768*2), inner_features2=500, out_features=n_classes)

In [0]:
from math import sqrt
class model_with_att(torch.nn.Module):
  def __init__(self, embedding_dim, out_GRu_feat, n):
        
        super().__init__()
        
        self.out_GRu_feat = out_GRu_feat

        self.out_biGRu = int(self.out_GRu_feat * 2)
        self.out_CNN = int(self.out_GRu_feat * 3)
        self.n = int(n)
        self.embedding_dim = embedding_dim

        self.drop_emb = torch.nn.Dropout(0.3)

        self.LSTM = torch.nn.LSTM(self.embedding_dim, out_GRu_feat, num_layers = 2, dropout = 0.4, bidirectional=True, batch_first=True) 

        #attention
        
        self.q_proj = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.k_proj = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.v_proj = torch.nn.Linear(self.out_biGRu, self.out_biGRu)
        self.att_soft = torch.nn.Softmax(dim = 2)
        
        self.cnn_3gr = torch.nn.Conv1d(in_channels=int(self.out_biGRu*2) , out_channels=self.out_GRu_feat, kernel_size=3)
        self.cnn_4gr = torch.nn.Conv1d(in_channels=int(self.out_biGRu*2) , out_channels=self.out_GRu_feat, kernel_size=4)
        self.cnn_5gr = torch.nn.Conv1d(in_channels=int(self.out_biGRu*2) , out_channels=self.out_GRu_feat, kernel_size=5)

        self.bn1 = torch.nn.BatchNorm1d(self.out_CNN)
        
        self.linear_1 = torch.nn.Linear(in_features=self.out_CNN, out_features=self.out_biGRu)
        gl = sqrt(6./(self.out_CNN+self.out_biGRu))
        self.linear_1.weight.data.uniform_(-gl, gl)

        self.relu = torch.nn.ReLU()

        self.drop_out = torch.nn.Dropout(0.2)

        self.bn2 = torch.nn.BatchNorm1d(self.out_biGRu)

        self.linear_2 = torch.nn.Linear(in_features=self.out_biGRu, out_features=n)
        gl2 = sqrt(6./(self.out_biGRu+n))
        self.linear_2.weight.data.uniform_(-gl2, gl2)

        
  def forward(self, x, x_len):
      

      x_emb = self.drop_emb(x)

      x_packed = pack_padded_sequence(x_emb, x_len, batch_first=True, enforce_sorted=False)

      x_lstm, _ = self.LSTM(x_packed)

      x, _ = pad_packed_sequence(x_lstm, batch_first=True)

      
      x_q = self.q_proj(x)
      x_k = self.k_proj(x)
      x_v = self.v_proj(x)

      att_scores = torch.bmm(x_q, x_k.transpose(2,1))/self.out_GRu_feat
      att_dist = self.att_soft(att_scores)
      attention_vectors = torch.bmm(att_dist, x_v)
      hidden_with_attention = torch.cat([x_v, attention_vectors], dim=-1)
     
      x_gru_s_t = hidden_with_attention.transpose(2,1)

      x_cnn3 = self.cnn_3gr(x_gru_s_t)
      x_cnn4 = self.cnn_4gr(x_gru_s_t)
      x_cnn5 = self.cnn_5gr(x_gru_s_t)

      frst, _ =  x_cnn3.max(dim= -1,)
      sc, _ = x_cnn4.max(dim= -1,)
      thr, _ = x_cnn5.max(dim= -1,)
      
      px = torch.cat((frst, sc, thr), dim=-1)
      
      x = self.bn1(px)
      x = self.linear_1(x)
      x = self.relu(x)
      x = self.drop_out(x)
      x = self.bn2(x)
      x = self.linear_2(x)
      x = self.relu(x)
    
      return x

In [0]:
model_att = model_with_att(768, 300, n_classes)

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

In [0]:
with torch.no_grad():
    pred = model(x, ones)

In [0]:
pred[0][:, 0 ,:].shape

torch.Size([512, 768])

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=mlp.parameters())

In [0]:
mlp = mlp.to(device)
model = model.to(device)
criterion = criterion.to(device)

In [0]:
mlp(x).shape

torch.Size([512, 28])

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence

epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y, x_len, ones in train_loader:


        x_len = x_len.to(device)
        x = x.to(device)
        y = y.to(device)
        ones = ones.to(device)


        optimizer.zero_grad()
        with torch.no_grad():
          x, _ = model(x, ones)
        x = x[:, 0 ,:]

        pred = mlp(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y, x_len, ones in validation_loader:
        
        
        x = x.to(device)
        x_len = x_len.to(device)
        y = y.to(device)
        ones = ones.to(device)

        with torch.no_grad():
            x, _ = model(x, ones)
            x = x[:, 0 ,:]
            pred = mlp(x)

            pred = pred.cpu()
            y = y.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 2:   0%|          | 0/599999 [00:00<?, ?it/s]


Losses: train - 2.597, test - 2.287
F1 test - 0.350


Epoch 3:   0%|          | 0/599999 [00:00<?, ?it/s]


Losses: train - 2.411, test - 2.209
F1 test - 0.370


Epoch 4:   0%|          | 0/599999 [00:00<?, ?it/s]

...

Epoch 5:   0%|          | 0/599999 [00:00<?, ?it/s]


Losses: train - 2.346, test - 2.152
F1 test - 0.385


Epoch 6:   0%|          | 0/599999 [00:00<?, ?it/s]


Losses: train - 2.328, test - 2.139
F1 test - 0.391


Epoch 6:  96%|█████████▌| 573952/599999 [1:08:09<03:05, 140.47it/s, train_loss=2.31]

In [0]:
model.eval()

predictions = []

for x, y, x_len,ones in test_loader:

    x = x.to(device)

    ones = ones.to(device)
    with torch.no_grad():

        x, _ = model(x, ones)
        pred = model_att(x,x_len)

        pred = pred.cpu()
        
        predictions.append(np.argmax(pred, axis=1))
        
predictions = np.concatenate(predictions).squeeze()

Epoch 5:   8%|▊         | 45568/599999 [04:50<56:45, 162.81it/s, train_loss=1.64]

In [0]:
test['main_category'] = predictions

In [0]:
test = test[['index', 'main_category']]

In [0]:
test.head()

,index,main_category
0,0,15
1,1,15
2,2,18
3,3,17
4,4,12


In [0]:
test.to_csv('submission11.csv', index=False)

In [0]:
from google.colab import files
files.download('submission11.csv')